In [1]:
from ultralytics import YOLO
import os
import torch
import yaml  
from pathlib import Path


# Configuration Settings
## Path to your dataset configuration file

In [ ]:
DATA_PATH = (
    r"Road-Anomaly-Detection\dataset\data.yaml"
)

# Model selection


In [3]:
MODEL_SIZE = "yolov8m.pt"


# Training settings


In [ ]:
TRAINING_ROUNDS = 120  
IMAGE_SIZE = 640  # input images size
BATCH_SIZE = 9  
PATIENCE = 25  # stopes if no improvement
SAVE_FOLDER = "RoadDetectionModel"
RUN_NAME = f"RoadModel_{MODEL_SIZE}_rounds{TRAINING_ROUNDS}_b{BATCH_SIZE}" 



In [ ]:
def check_system_resources():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)

        print(f"✓ GPU detected: {gpu_name}")
        print(f"✓ GPU memory: {memory_gb:.1f} GB")

        if memory_gb < 4:
            print(f"⚠ Low GPU memory detected (<4GB). Recommended batch size: 2-4.")
            if BATCH_SIZE > 4:
                print(f"‼ Your current batch size ({BATCH_SIZE}) is likely TOO HIGH.")
        elif memory_gb < 8:
            print(
                f"ℹ Medium GPU memory detected ({memory_gb:.1f}GB). Recommended max batch size: 8-12 for yolov8m."
            )
            if BATCH_SIZE > 12:
                print(f"‼ Your current batch size ({BATCH_SIZE}) might be too high.")
            elif BATCH_SIZE > 8:
                print(
                    f"⚠ Your current batch size ({BATCH_SIZE}) is ambitious. Monitor memory usage."
                )
        else:
            print(f"✓ Sufficient GPU memory for selected batch size ({BATCH_SIZE})")
    else:
        print("⚠ No GPU detected - training will be very slow on CPU")
        if BATCH_SIZE > 4:
            print("⚠ Reduce batch size to 2-4 for CPU training")



In [ ]:
def display_results(metrics, class_names):
    print("\n--- Model Performance Summary (Test Set) ---")

    print(f"\nOverall Detection Metrics:")
    print(
        f"  ├─ mAP@.5:.95: {metrics.box.map:.1%}"
    ) 
    print(
        f"  └─ mAP@.5:    {metrics.box.map50:.1%}"
    )

    # Per-class metrics
    print(f"\nPer-Class Performance (mAP):")
    if len(class_names) == len(metrics.box.maps):
        for i, class_name in enumerate(class_names.values()):
            map50_95 = metrics.box.maps[i]
            map50 = metrics.box.ap50[i]  # ap50 contains the per-class mAP@0.5 scores
            print(f"  {class_name}:")
            print(f"    ├─ mAP@.5:.95: {map50_95:.1%}")
            print(f"    └─ mAP@.5:    {map50:.1%}")
    else:
        print(
            "Warning: Number of class names doesn't match number of metrics returned."
        )
        print("Raw maps:", metrics.box.maps)
        print("Raw ap50:", metrics.box.ap50)

In [ ]:
def main():
    print("\n=== Road Defect Detection Model Training ===\n")

    data_path_obj = Path(DATA_PATH)
    if not data_path_obj.is_file():
        print(f"❌ Error: Dataset configuration file not found at {DATA_PATH}")
        print("   Please check the file path and try again.")
        return

    check_system_resources()


    print(f"\nLoading model: {MODEL_SIZE}")
    model = YOLO(MODEL_SIZE)

    print(f"\nStarting training:")
    print(f"  - Dataset Config: {DATA_PATH}")
    print(f"  - Training Rounds: {TRAINING_ROUNDS}")
    print(f"  - Image Size: {IMAGE_SIZE}")
    print(f"  - Batch Size: {BATCH_SIZE}")
    print(f"  - Patience: {PATIENCE}")
    print(f"  - Saving to: {SAVE_FOLDER}/{RUN_NAME}")

    results = model.train(
        data=str(data_path_obj), 
        epochs=TRAINING_ROUNDS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        patience=PATIENCE,
        project=SAVE_FOLDER,
        name=RUN_NAME,
        exist_ok=False, 
    )

    save_dir = Path(results.save_dir)
    print(f"\n✓ Training complete!")
    print(f"✓ Results saved to: {save_dir}")

    print("\nEvaluating model on test images...")
    best_model_path = save_dir / "weights" / "best.pt"
    if not best_model_path.is_file():
        print(f"❌ Error: Best model weights not found at {best_model_path}")
        return

    best_model = YOLO(str(best_model_path))

    try:
        metrics = best_model.val(
            split="test", data=str(data_path_obj)
        )  
        class_names_from_model = (
            best_model.names if hasattr(best_model, "names") else {}
        )
        display_results(metrics, class_names_from_model)
    except Exception as e:
        print(f"❌ Error during evaluation: {e}")

    print(f"\nModel training and evaluation complete!")
    print(f"Your best trained model is saved at: {best_model_path}")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nTraining stopped by user. Any saved weights can still be used.")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")



=== Road Defect Detection Model Training ===

✓ GPU detected: NVIDIA GeForce RTX 3060 Laptop GPU
✓ GPU memory: 6.0 GB
ℹ Medium GPU memory detected (6.0GB). Recommended max batch size: 8-12 for yolov8m.
⚠ Your current batch size (9) is ambitious. Monitor memory usage.

Loading model: yolov8m.pt

Starting training:
  - Dataset Config: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\data.yaml
  - Training Rounds: 120
  - Image Size: 640
  - Batch Size: 9
  - Patience: 25
  - Saving to: RoadDetectionModel/RoadModel_yolov8m.pt_rounds120_b9
New https://pypi.org/project/ultralytics/8.3.118 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.116  Python-3.9.21 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\data.yaml, epochs=120, time=None, patience=25, batch=9, imgsz=640, save=True, save_period=-

100%|██████████| 5.35M/5.35M [00:24<00:00, 233kB/s] 


AMP: checks passed 
train: Fast image access  (ping: 0.20.0 ms, read: 446.9339.0 MB/s, size: 1020.2 KB)


train: Scanning C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\labels.cache... 23321 images, 0 backgrounds, 13 corrupt: 100%|██████████| 23321/23321 [00:00<?, ?it/s]

train: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\images\dataset4_DJI_20230707094615_0034_640_02_05.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0016]
train: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\images\dataset4_DJI_20230707094904_0193_640_00_04.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0016]
train: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\images\dataset4_DJI_20230707094956_0244_640_07_06.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
train: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\images\dataset4_DJI_20230707095024_0271_640_04_05.jpg: ignoring corrupt image/label: negative label values [   -0.15313]
train: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\train\images\dataset4_DJI_20230

...

val: Scanning C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\test\labels... 2846 images, 0 backgrounds, 2 corrupt: 100%|██████████| 2846/2846 [00:07<00:00, 386.22it/s]

val: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\test\images\dataset4_DJI_20230707094940_0228_640_02_07.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0063]
val: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\test\images\dataset4_DJI_20230707095001_0248_640_05_07.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0047]


val: New cache created: C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\test\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 178/178 [00:53<00:00,  3.34it/s]


                   all       2844       7696      0.736       0.74      0.745      0.447
         Heavy-Vehicle        504        647      0.913      0.978      0.981      0.763
         Light-Vehicle        928       2158      0.892      0.951      0.961      0.649
            Pedestrian        317        561      0.822      0.915      0.918      0.522
                 Crack       1284       2413      0.576      0.484      0.505       0.24
          Crack-Severe        722       1005      0.548      0.503      0.493      0.273
               Pothole        347        771      0.595      0.438      0.468      0.199
            Speed-Bump        135        141      0.804      0.908      0.885      0.485
Speed: 0.2ms preprocess, 11.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\val

--- Model Performance Summary (Test Set) ---

Overall Detection Metrics:
  ├─ mAP@.5:.95: 44.7%
  └─ mAP@.5:    74.5%

Per-Class Performance (mAP):
  Heavy-Vehicle:
    ├─

In [10]:
from pathlib import Path
from ultralytics import YOLO
from ultralytics.utils.metrics import ConfusionMatrix

MODEL_PATH = r".\RoadDetectionModel\RoadModel_yolov8m.pt_rounds120_b9\weights\best.pt"
DATA_YAML_PATH = r".\dataset\data.yaml"

def evaluate_on_test():
    print("\n=== Evaluating YOLOv8 Model on Test Set ===")

    model_path = Path(MODEL_PATH)
    if not model_path.is_file():
        print(f"❌ Best model not found at {model_path}")
        return

    model = YOLO(str(model_path))

    print(f"✅ Loaded model: {model_path.name}")

    metrics = model.val(split="test", data=DATA_YAML_PATH)

    precision = metrics.box.p[0]
    recall = metrics.box.r[0]
    map50 = metrics.box.map50
    map_val = metrics.box.map

    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)

    print("\n📊 Test Set Evaluation Metrics:")
    print(f"  - Precision:      {precision:.4f}")
    print(f"  - Recall:         {recall:.4f}")
    print(f"  - mAP@0.5:        {map50:.4f}")
    print(f"  - mAP@0.5:0.95:   {map_val:.4f}")
    print(f"  - F1-score:       {f1_score:.4f}")
    print(f"📂 Results saved in: {metrics.save_dir}")

if __name__ == "__main__":
    evaluate_on_test()


=== Evaluating YOLOv8 Model on Test Set ===


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000028850992A60>
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\tf\lib\site-packages\torch\utils\data\dataloader.py", line 1663, in __del__
    self._shutdown_workers()
  File "c:\ProgramData\anaconda3\envs\tf\lib\site-packages\torch\utils\data\dataloader.py", line 1621, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


✅ Loaded model: best.pt
Ultralytics 8.3.120  Python-3.9.21 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 92 layers, 25,843,813 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 670.5197.7 MB/s, size: 215.1 KB)


val: Scanning C:\Users\Navneet Sharma\Documents\GitHub\Road-Anomaly-Detection\dataset\test\labels.cache... 2844 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2844/2844 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 178/178 [00:59<00:00,  3.01it/s]


                   all       2844       7696      0.736       0.74      0.745      0.448
         Heavy-Vehicle        504        647      0.913      0.978      0.981      0.763
         Light-Vehicle        928       2158      0.892      0.951      0.961      0.649
            Pedestrian        317        561      0.822      0.915      0.918      0.522
                 Crack       1284       2413      0.576      0.484      0.505       0.24
          Crack-Severe        722       1005      0.548      0.503      0.493      0.273
               Pothole        347        771      0.597       0.44      0.468      0.198
            Speed-Bump        135        141      0.804      0.908      0.885      0.487
Speed: 0.2ms preprocess, 12.0ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\val3

📊 Test Set Evaluation Metrics:
  - Precision:      0.9128
  - Recall:         0.9784
  - mAP@0.5:        0.7447
  - mAP@0.5:0.95:   0.4476
  - F1-score:       0.9445
📂 Re